In [ ]:
import pandas as pd
import seaborn as sns
from glob import glob
sns.set_theme(style="ticks", font="Linux Biolinum O", font_scale=1.2)

frames = [pd.read_csv(data) for data in glob('../data/operation-*.txt')]
data = pd.concat(frames, ignore_index=True)
data = data[data['comment'].isna()]
data = data[(data['num_concurrent'] == 1) | (data['operation'] == 'repair') & (data['num_concurrent'] == 2)]
data["outer_coding"] = list(zip(data["outer_n"], data["outer_k"]))
data["inner_coding"] = list(zip(data["inner_n"], data["inner_k"]))

def protocol_operation(row):
    return f'{row["protocol"]} - {row["operation"]}'

data["protocol - operation"] = data.apply(protocol_operation, axis=1)
data

In [ ]:
hue_order = ["entropy - put", "kademlia - put", "entropy - get", "kademlia - get", "entropy - repair", "kademlia - repair"]
entropy_palette = sns.cubehelix_palette(n_colors=6)
kademlia_palette = sns.cubehelix_palette(start=2., rot=0.4, n_colors=6)
palette = [c for p in zip(entropy_palette, kademlia_palette) for c in p]

ax = sns.catplot(data=data[data["inner_coding"] == (80, 32)], 
    kind="bar",
    x="outer_coding", y="latency", hue="protocol - operation",
    order=[(5, 4), (10, 8), (15, 12)],
    hue_order=hue_order, palette=palette,
    # legend=False,
    height=2.56, aspect=2.)
# ax.add_legend(label_order = ["entropy - put", "entropy - get", "entropy - repair", "kademlia - put", "kademlia - get", "kademlia - repair"])
ax \
.set_xlabels("Outer Code") \
.set_ylabels("Latency (s)") \
.savefig("operation-outer.pdf")

In [ ]:
sns.catplot(data=data[data["outer_coding"] == (10, 8)], 
    kind="bar",
    x="inner_coding", y="latency", hue="protocol - operation",
    order=[(40, 16), (80, 32), (120, 48)],
    hue_order=hue_order, palette=palette,
    height=2.56, aspect=2.) \
.set_xlabels("Inner Code") \
.set_ylabels("Latency (s)") \
.savefig("operation-inner.pdf")